In [266]:
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt

from datetime import datetime 
from collections import defaultdict
from scipy.stats import pearsonr

In [267]:
#create a pandas dataframes from csv files
users = pd.read_csv('users.csv')
tweets = pd.read_csv('tweets.csv')

In [268]:
users.info()
users.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11508 entries, 0 to 11507
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              11508 non-null  int64  
 1   name            11507 non-null  object 
 2   lang            11508 non-null  object 
 3   bot             11508 non-null  int64  
 4   created_at      11508 non-null  object 
 5   statuses_count  11109 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 539.6+ KB


id                  int64
name               object
lang               object
bot                 int64
created_at         object
statuses_count    float64
dtype: object

In [269]:
tweets.info()
tweets.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13664696 entries, 0 to 13664695
Data columns (total 10 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   id              object
 1   user_id         object
 2   retweet_count   object
 3   reply_count     object
 4   favorite_count  object
 5   num_hashtags    object
 6   num_urls        object
 7   num_mentions    object
 8   created_at      object
 9   text            object
dtypes: object(10)
memory usage: 1.0+ GB


id                object
user_id           object
retweet_count     object
reply_count       object
favorite_count    object
num_hashtags      object
num_urls          object
num_mentions      object
created_at        object
text              object
dtype: object

In [270]:
tweets['id'] = pd.to_numeric(tweets['id'], errors='coerce')
tweets['user_id'] = pd.to_numeric(tweets['user_id'], errors='coerce')
tweets['retweet_count'] = pd.to_numeric(tweets['retweet_count'], errors='coerce')
tweets['reply_count'] = pd.to_numeric(tweets['reply_count'], errors='coerce')
tweets['favorite_count'] = pd.to_numeric(tweets['favorite_count'], errors='coerce')
tweets['num_hashtags'] = pd.to_numeric(tweets['num_hashtags'], errors='coerce')
tweets['num_mentions'] = pd.to_numeric(tweets['num_mentions'], errors='coerce')
tweets['num_urls'] = pd.to_numeric(tweets['num_urls'], errors='coerce')
tweets['created_at'] = pd.to_datetime(tweets['created_at'], errors='coerce')

In [271]:
tweets.info()
tweets.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13664696 entries, 0 to 13664695
Data columns (total 10 columns):
 #   Column          Dtype         
---  ------          -----         
 0   id              float64       
 1   user_id         float64       
 2   retweet_count   float64       
 3   reply_count     float64       
 4   favorite_count  float64       
 5   num_hashtags    float64       
 6   num_urls        float64       
 7   num_mentions    float64       
 8   created_at      datetime64[ns]
 9   text            object        
dtypes: datetime64[ns](1), float64(8), object(1)
memory usage: 1.0+ GB


id                       float64
user_id                  float64
retweet_count            float64
reply_count              float64
favorite_count           float64
num_hashtags             float64
num_urls                 float64
num_mentions             float64
created_at        datetime64[ns]
text                      object
dtype: object

In [272]:
#tweets.dropna(inplace=True)

In [273]:
tweets.drop_duplicates(inplace=True)
#tweets.drop_duplicates(subset=['id'], inplace=True)

In [274]:
tweets.info()
tweets.dtypes

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11712595 entries, 0 to 13664695
Data columns (total 10 columns):
 #   Column          Dtype         
---  ------          -----         
 0   id              float64       
 1   user_id         float64       
 2   retweet_count   float64       
 3   reply_count     float64       
 4   favorite_count  float64       
 5   num_hashtags    float64       
 6   num_urls        float64       
 7   num_mentions    float64       
 8   created_at      datetime64[ns]
 9   text            object        
dtypes: datetime64[ns](1), float64(8), object(1)
memory usage: 983.0+ MB


id                       float64
user_id                  float64
retweet_count            float64
reply_count              float64
favorite_count           float64
num_hashtags             float64
num_urls                 float64
num_mentions             float64
created_at        datetime64[ns]
text                      object
dtype: object

In [275]:
#aggiungere per ultimi 3 attributi
#eliminazione valori negativi
tweets = tweets[tweets['retweet_count'] >= 0]
tweets = tweets[tweets['reply_count'] >= 0]
tweets = tweets[tweets['favorite_count'] >= 0]
tweets.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10297540 entries, 0 to 13664695
Data columns (total 10 columns):
 #   Column          Dtype         
---  ------          -----         
 0   id              float64       
 1   user_id         float64       
 2   retweet_count   float64       
 3   reply_count     float64       
 4   favorite_count  float64       
 5   num_hashtags    float64       
 6   num_urls        float64       
 7   num_mentions    float64       
 8   created_at      datetime64[ns]
 9   text            object        
dtypes: datetime64[ns](1), float64(8), object(1)
memory usage: 864.2+ MB


In [276]:
tweets.sort_values('favorite_count').head(50)

,id,user_id,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions,created_at,text
0,5.093540e+17,3.277463e+08,0.0,0.0,0.0,0.0,0.0,0.0,2019-09-11 14:53:55,"If man is a little lower than angels, then ang..."
7980306,2.691764e+17,6.162257e+08,0.0,0.0,0.0,1.0,0.0,0.0,2017-11-17 20:33:55,overcapacity#overcapacity thatsJustStupid haha
7980303,2.974418e+17,1.423570e+07,0.0,0.0,0.0,0.0,1.0,0.0,2018-02-03 20:30:22,Landing in about 30 min (@ Hartsfield-Jackson...
7980302,5.298411e+17,1.950145e+08,0.0,0.0,0.0,0.0,0.0,0.0,2019-11-07 03:42:11,I won't be a rock star. I will be a legend. - ...
7980301,5.858716e+17,3.101627e+09,0.0,0.0,0.0,0.0,0.0,1.0,2020-04-09 18:27:17,@MysidianRabbit hahah it's me Joy ^^ my primar...
7980299,5.548018e+17,2.843210e+08,0.0,0.0,0.0,2.0,1.0,2.0,2020-01-15 00:47:08,3 Challenges Faced by Enterprise SEOs in 2015 ...
7980298,5.931803e+17,2.600379e+08,9.0,0.0,0.0,1.0,0.0,2.0,2020-04-29 22:29:28,RT @wfaachannel8: #BREAKING School bus crash r...
7980297,5.637568e+17,1.324797e+09,0.0,0.0,0.0,1.0,0.0,0.0,2020-02-08 17:51:17,is jinhwan really a 94 liner? He looks like 12...
7980295,5.764310e+17,3.292850e+08,5.0,0.0,0.0,0.0,1.0,1.0,2020-03-14 17:13:48,"RT @JenChancellor: This bigoted, racist shit i..."
7980294,5.166333e+17,5.608727e+08,2.0,0.0,0.0,0.0,0.0,0.0,2019-10-01 16:59:10,"""Essere completamente onesti con se stessi Ã¨ ..."


In [277]:
tweets['retweet_count'].quantile(0.75) + 1.5*(tweets['retweet_count'].quantile(0.75)-tweets['retweet_count'].quantile(0.25))

2.5

In [278]:
tweets.replace(to_replace='inf', value=0, inplace=True)
tweets.sort_values('retweet_count', ascending=False)

,id,user_id,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions,created_at,text
1606076,9.613119e+08,2.374871e+09,2.000000e+134,0.0,0.0,NaN,NaN,NaN,2019-07-06 10:54:48,NaN
3841610,1.690871e+16,4.653922e+08,7.000000e+84,0.0,0.0,NaN,0.0,0.0,2019-11-03 22:29:44,La mamma Ã¨ quella che ti insegna a pregare--P...
4727864,7.475156e+08,2.516336e+08,6.000000e+77,0.0,0.0,0.0,0.0,NaN,2020-04-07 06:38:50,RT @AlexDijkstra_: @tapijtlegger @MDMAbleToShi...
12849756,6.684369e+08,2.359389e+09,7.000000e+71,0.0,0.0,0.0,NaN,0.0,2020-04-12 11:31:44,TBH THE ONLY DRAMA WITH A GOOD ENDING
8432312,6.486533e+16,3.929634e+08,4.000000e+65,0.0,0.0,NaN,0.0,1.0,2020-03-02 05:31:18,@tiamojai he's actually sweet as pie to his bm...
...,...,...,...,...,...,...,...,...,...,...
4705968,5.707200e+17,8.762971e+08,0.000000e+00,0.0,0.0,0.0,0.0,0.0,2020-02-27 23:00:29,http://t.co/srVkJIaBo8
4705967,6.349990e+16,4.890426e+08,0.000000e+00,0.0,1.0,0.0,0.0,1.0,2040-03-12 06:59:19,@juustlikekamii P0RN!!1!1!1!
4705966,5.868563e+17,4.383133e+07,0.000000e+00,0.0,0.0,0.0,0.0,1.0,2020-04-12 11:40:09,@squid_exe sleazy instead of provocative
4705965,4.358238e+17,2.276558e+09,0.000000e+00,0.0,1.0,0.0,0.0,0.0,2019-02-20 17:11:10,"I grandi Dubbi!! PerchÃ¨ si chiama contagocce,..."


In [279]:
retweet_clean = tweets[tweets['retweet_count'] != 0]['retweet_count']
reply_clean = tweets[tweets['reply_count'] != 0]['reply_count']
favorite_clean = tweets[tweets['favorite_count'] != 0]['favorite_count']
hashtags_clean = tweets[tweets['num_hashtags'] != 0]['num_hashtags']
urls_clean = tweets[tweets['num_urls'] != 0]['num_urls']
mentions_clean = tweets[tweets['num_mentions'] != 0]['num_mentions']

In [280]:
#retweet_clean.quantile(0.75) + 1.5*(retweet_clean.quantile(0.75)-retweet_clean.quantile(0.25))

In [281]:
#retweet_clean.median()

In [282]:
#retweet_clean.plot.box()
#plt.yscale('log')

In [283]:
#decidere se togliere il valore anomalo 
#retweet_clean = retweet_clean.to_frame()
#retweet_clean2 = retweet_clean[retweet_clean['retweet_count'] < 4000000]['retweet_count']

In [284]:
#retweet_clean2.quantile(0.75) + 1.5*(retweet_clean2.quantile(0.75)-retweet_clean2.quantile(0.25))

In [285]:
#retweet_clean2.median()

In [286]:
#retweet_clean2.plot.box()
#plt.yscale('log')

In [287]:
#favorite_clean.quantile(0.75) + 1.5*(favorite_clean.quantile(0.75)-favorite_clean.quantile(0.25))

In [288]:
#decidere se togliere il valore anomalo 
#favorite_clean2 = favorite_clean.to_frame()
#favorite_clean2 = favorite_clean2[favorite_clean2['favorite_count'] < 4000000]['favorite_count']

In [289]:
#favorite_clean2.plot.box()
#plt.yscale('log')

In [290]:
#favorite_clean2.mad(axis = 0, skipna = True)


In [291]:
#retweet_clean.mad(axis = 0, skipna = True)


In [292]:
#retweet_clean2.mad(axis = 0, skipna = True)


In [293]:
#clean = tweets
#clean['retweet_count'] = tweets[tweets['retweet_count'] != 0]['retweet_count']
#clean['retweet_count'] = clean[clean['retweet_count'] < 4000000]['retweet_count']
#clean['reply_count'] = tweets[tweets['reply_count'] != 0]['reply_count']
#clean['favorite_count'] = tweets[tweets['favorite_count'] != 0]['favorite_count']
#clean['favorite_count'] = clean[clean['favorite_count'] < 4000000]['favorite_count']
#clean['num_hashtags'] = tweets[tweets['num_hashtags'] != 0]['num_hashtags']
#clean['num_urls'] = tweets[tweets['num_urls'] != 0]['num_urls']
#clean['num_mentions'] = tweets[tweets['num_mentions'] != 0]['num_mentions']

#clean.info()

In [294]:
#clean.boxplot(column=['retweet_count', 'reply_count', 'favorite_count'])
#plt.yscale('log')

In [295]:
#clean['reply_count'].median()

In [296]:
#clean['reply_count'].quantile(0.75)

In [297]:
#limit_reply = clean['reply_count'].quantile(0.75) + 1.5*(clean['reply_count'].quantile(0.75)-clean['reply_count'].quantile(0.25))
#clean[(clean['reply_count'] > limit_reply)]['reply_count'].count()

In [298]:
#limit_retweet = clean['retweet_count'].quantile(0.75) + 1.5*(clean['retweet_count'].quantile(0.75)-clean['retweet_count'].quantile(0.25))
#clean[(clean['retweet_count'] > limit_retweet)]['retweet_count'].count()

In [299]:
#limit_favorite = clean['favorite_count'].quantile(0.75) + 1.5*(clean['favorite_count'].quantile(0.75)-clean['favorite_count'].quantile(0.25))
#clean[(clean['favorite_count'] > limit_favorite)]['favorite_count'].count()

In [300]:
user_median = tweets.groupby('user_id').median()

/var/folders/h4/5wp1mt_n1k72n6zcf7wxd6bc0000gn/T/ipykernel_20434/2198640111.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  user_median = tweets.groupby('user_id').median()


In [301]:
user_median.sort_values('retweet_count', ascending=False).head(50)


,id,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions
user_id,,,,,,,
2.704427e+08,2.541554e+17,129468.0,16.0,16.0,1.0,0.0,1.0
1.828710e+05,NaN,58211.0,0.0,0.0,NaN,NaN,NaN
2.430000e+02,7.460479e+10,25626.0,0.0,0.0,NaN,NaN,NaN
2.519938e+08,2.815094e+17,17642.0,19.0,34.0,0.0,1.0,1.0
5.468187e+08,2.886579e+17,14433.0,26.0,22.0,0.0,1.0,1.0
2.563759e+08,2.926623e+17,9669.0,2.0,8.5,0.0,0.5,1.0
6.700000e+01,3.505997e+14,9630.0,0.0,0.0,0.0,NaN,1.0
7.180000e+02,3.516383e+08,8307.0,0.0,0.0,NaN,2.0,1.0
1.418864e+08,3.271155e+17,6782.0,2.0,1.0,0.0,1.0,1.0


In [302]:
#df = user_median.where(df.isna() & user_median['user_id'] == df['user_id'])

In [303]:

#user_median['id'] = user_median.index

In [304]:

df1 = user_median.reset_index()
df1.columns

Index(['user_id', 'id', 'retweet_count', 'reply_count', 'favorite_count',
       'num_hashtags', 'num_urls', 'num_mentions'],
      dtype='object')

In [305]:

df = tweets.merge(df1, how='left', on='user_id')

In [306]:
df['retweet_count_x'].isna().count()

10297540

In [307]:
df['retweet_count_x'].fillna(df['retweet_count_y'][df.index], inplace=True)

In [313]:
df['retweet_count_x'].isna()

10297540

In [309]:
user_median['retweet_count'].isna().sum()

0

In [310]:
df.sort_values('retweet_count_x')
#df['retweet_count_x'].quantile([.25,.5,.75])
tweets['retweet_count'].isna().sum()

0

In [311]:
wrong = tweets[tweets['num_hashtags'].isna() & tweets['user_id'].notna()]
wrong.head(10)

,id,user_id,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions,created_at,text
32,NaN,2.214453e+09,1.0,0.0,2.0,NaN,NaN,NaN,2019-01-18 01:13:23,NaN
347,8.669884e+09,1.888349e+09,0.0,0.0,1.0,NaN,0.0,NaN,2020-03-31 01:19:27,SAM SMITH RN
411,NaN,4.662345e+08,0.0,0.0,0.0,NaN,0.0,0.0,2019-11-02 18:16:33,L'aspirina e' un potente anticoncezionale. Bas...
419,NaN,4.663959e+08,0.0,0.0,3.0,NaN,0.0,0.0,2019-08-12 20:07:21,"-Ciao amore, come sto senza trucco? -E TU CHI ..."
517,3.757205e+15,1.970830e+09,0.0,0.0,0.0,NaN,NaN,NaN,2019-06-19 21:55:22,NaN
647,NaN,6.790029e+07,0.0,0.0,0.0,NaN,0.0,1.0,2018-01-24 16:50:29,@JamalLeghari Then Election Or What ??
653,NaN,6.496532e+07,0.0,0.0,1.0,NaN,NaN,NaN,2018-01-31 02:21:28,NaN
712,NaN,3.862418e+08,5.0,0.0,0.0,NaN,NaN,NaN,2020-01-30 04:00:09,NaN
758,6.243894e+17,4.661947e+08,0.0,0.0,0.0,NaN,0.0,0.0,2019-10-31 14:36:46,"""Non Ã¨ facile badare alla convenienza quando ..."
781,4.538975e+17,1.403564e+08,0.0,0.0,0.0,NaN,0.0,1.0,2020-04-02 06:22:01,@AkshayRules kya rate chlre uske aajkl?


In [312]:
user_median.rename(columns= {'id':'user_id'})
wrong['num_hashtags'] = user_median['num_hashtags'][user_median['user_id'] == wrong.user_id]

AttributeError: 'DataFrame' object has no attribute 'user_id'